## init

In [ ]:
!nvidia-smi

Sun Nov 23 19:04:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd {HOME}/segment-anything-2
!pip install -e . -q
!pip install -q supervision jupyter_bbox_widget
!mkdir -p {HOME}/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt -P {HOME}/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt -P {HOME}/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt -P {HOME}/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -P {HOME}/checkpoints
!pip install boto3 sentence-transformers

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (381/381), done.
/content/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.9 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.6 MB/s eta 0:00:00
   ━━━━

## init 2

In [ ]:
!pip install ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.4 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
import cv2
import torch
import base64

import numpy as np
import supervision as sv

from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
# !mkdir -p {HOME}/checkpoints
# !wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt -P {HOME}/checkpoints
# !wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt -P {HOME}/checkpoints
# !wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt -P {HOME}/checkpoints
# !wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -P {HOME}/checkpoints

## Main code


In [ ]:
# main
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import base64
import cv2
import numpy as np
import io
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import supervision as sv
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import uuid
import asyncio
from concurrent.futures import ThreadPoolExecutor
import time
from openai import OpenAI  # Changed from import openai
import logging
import re

In [ ]:
# main
import boto3
class S3ImageStorageManager:
    """Manages image storage in AWS S3"""

    def __init__(self, bucket_name: str, aws_access_key_id: str,
                 aws_secret_access_key: str, region_name: str = 'us-east-1'):
        """Initialize S3 client"""
        self.s3_client = boto3.client(
            's3',
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            region_name=region_name
        )
        self.bucket_name = bucket_name
        self.region_name = region_name

    def upload_base64_image(self, base64_image: str, prefix: str = 'frames/') -> str:
        """
        Upload base64 encoded image to S3 and return public URL

        Returns:
        - Public S3 URL to access the image
        """
        try:
            # Decode base64 image
            image_bytes = base64.b64decode(base64_image)

            # Generate unique filename with timestamp
            timestamp = int(time.time())
            filename = f"{prefix}{timestamp}_{uuid.uuid4()}.png"

            # Upload to S3 with public-read ACL
            self.s3_client.put_object(
                Bucket=self.bucket_name,
                Key=filename,
                Body=image_bytes,
                ContentType='image/png',

            )

            # Generate public URL
            public_url = f"https://{self.bucket_name}.s3.{self.region_name}.amazonaws.com/{filename}"

            print(f"✅ Uploaded to S3: {filename}")
            return public_url

        except Exception as e:
            print(f"❌ Error uploading image to S3: {e}")
            raise

In [ ]:
#main
import requests

def store_in_image_analysis_db(
    patient_id: str,
    session_id: str,
    original_image_url: str,
    segmented_image_url: str,
    description: str,
    timestamp: float
) -> dict:
    """
    Store image analysis in database via API

    Returns:
    - API response with analysis_id
    """
    try:
        payload = {
            "patient_id": patient_id,
            "session_id": session_id,
            "image_type": "surgical_frame",  # You can make this dynamic
            "original_image_url": original_image_url,
            "segmented_image_url": segmented_image_url,
            "description": description,
            "timestamp": timestamp
        }

        response = requests.post(
            IMAGE_ANALYSIS_API,
            json=payload,
            timeout=10
        )

        if response.status_code == 201:
            result = response.json()
            analysis_id = result.get('result', {}).get('analysis_id', 'Unknown')
            print(f"✅ Stored in DB with analysis_id: {analysis_id}")
            return result
        else:
            print(f"⚠️ Failed to store in DB: {response.status_code}")
            print(f"   Response: {response.text}")
            raise Exception(f"Database storage failed: {response.text}")

    except Exception as e:
        print(f"❌ Error storing in database: {e}")
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
# main
import asyncio
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import sys
import subprocess
import threading
import time

# Apply nest_asyncio patch to allow nested event loops in Colab
nest_asyncio.apply()

def start_server_threaded(app, port, ngrok_token):
    """
    Start the server in a separate thread with Ngrok tunnel

    Args:
    - app: FastAPI application instance
    - port: Port to run the server on
    - ngrok_token: Ngrok authentication token
    """
    # Set Ngrok auth token
    ngrok.set_auth_token(ngrok_token)

    try:
        # Create ngrok tunnel
        public_url = ngrok.connect(port, proto="http")

        # Print connection details
        print(f"🌐 Public URL: {public_url}")
        print(f"\n✅ Server is ready!")
        print(f"\nUse this URL in Unity:")
        print(f"   {public_url}/segment")
        print(f"\nTest health check:")
        print(f"   {public_url}/health")

        # Prepare uvicorn configuration
        config = uvicorn.Config(app, host="0.0.0.0", port=port)
        server = uvicorn.Server(config)

        # Run the server
        asyncio.run(server.serve())

    except Exception as e:
        print(f"Error starting server: {e}")
    finally:
        # Disconnect all ngrok tunnels
        ngrok.kill()

def setup_and_run_server(app, ngrok_token, port=5556):
    """
    Comprehensive setup and server startup function for Colab

    Args:
    - app: FastAPI application instance
    - ngrok_token: Your Ngrok authentication token
    - port: Port to run the server on (default 5556)
    """
    # Install dependencies
    subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                           'fastapi', 'uvicorn', 'pyngrok',
                           'nest_asyncio', 'torch', 'transformers',
                           'opencv-python-headless', 'supervision'])

    # Apply nest_asyncio patch
    nest_asyncio.apply()


    server_thread = threading.Thread(
        target=start_server_threaded,
        args=(app, port, ngrok_token),
        daemon=True
    )
    server_thread.start()


    return server_thread

IMAGE_ANALYSIS_API = "https://jp758vdtak.us-east-1.awsapprunner.com/table/image_analysis"

# Simplified store_frame function - RDS only
def store_frame_data(
    patient_id: str,
    session_id: str,
    frame_base64: str,
    segmented_frame: str,
    caption: str
):
    """
    Store frame data in AWS RDS only:
    - Images uploaded to S3
    - All metadata and caption stored in RDS
    """
    try:
        # Upload images to S3
        original_frame_url = s3_manager.upload_base64_image(frame_base64, prefix='original/')
        segmented_frame_url = s3_manager.upload_base64_image(segmented_frame, prefix='segmented/')

        timestamp = time.time()

        # Store everything in AWS RDS
        db_result = store_in_image_analysis_db(
            patient_id=patient_id,
            session_id=session_id,
            original_image_url=original_frame_url,
            segmented_image_url=segmented_frame_url,
            description=caption,
            timestamp=timestamp
        )

        analysis_id = db_result.get('result', {}).get('analysis_id')
        print(f"✅ Stored in RDS with analysis_id: {analysis_id}")

        return {
            "analysis_id": analysis_id,
            "original_url": original_frame_url,
            "segmented_url": segmented_frame_url
        }

    except Exception as e:
        print(f"Storage error: {e}")
        raise HTTPException(status_code=500, detail=str(e))




In [ ]:
# main
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import base64
import cv2
import numpy as np
import io
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
import supervision as sv
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import uuid
import asyncio
from concurrent.futures import ThreadPoolExecutor
import time
from openai import OpenAI  # Changed from import openai
import logging
import re

# Initialize FastAPI app
app = FastAPI()

# Initialize models and embeddings

##################### UPDATE THESE PATHS!!!!!!! #####################################
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT = "/content/checkpoints/sam2_hiera_large.pt"
CONFIG = "sam2_hiera_l.yaml"



sam2_model = build_sam2(CONFIG, CHECKPOINT, device=DEVICE, apply_postprocessing=False)
sam2_model.eval()

mask_generator = SAM2AutomaticMaskGenerator(
    model=sam2_model,
    points_per_side=16,
    points_per_batch=128,
    pred_iou_thresh=0.6,
    stability_score_thresh=0.9,
    stability_score_offset=0.7,
    crop_n_layers=0,
    box_nms_thresh=0.7,
    min_mask_region_area=0,
    apply_postprocessing=False
)

mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)



# Shared Processing Pool
executor = ThreadPoolExecutor(max_workers=2)

# OpenAI client (global variable)
client = None

# Request model
class FrameRequest(BaseModel):
    frame_base64: str
    patient_id: str = "None" 
    session_id: str = None  
    frame_id: str = None  



def initialize_openai(api_key: str):
    """Initialize OpenAI client"""
    global client
    client = OpenAI(api_key=api_key)

def process_segment(frame_base64: str) -> dict:
    """Process segmentation for a frame"""
    try:
        # Decode the base64 image to a numpy array
        image_bytes = base64.b64decode(frame_base64)
        npimg = np.frombuffer(image_bytes, np.uint8)
        image_bgr = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

        # Generate segmentation mask
        sam2_result = mask_generator.generate(image_rgb)
        detections = sv.Detections.from_sam(sam_result=sam2_result)

        # Annotate the frame
        annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)

        # Encode back to base64
        _, buffer = cv2.imencode(".png", annotated_image)
        encoded_image = base64.b64encode(buffer).decode("utf-8")

        return {"segmented_frame": encoded_image}
    except Exception as e:
        print(f"Segmentation error: {e}")
        raise HTTPException(status_code=500, detail=str(e))


def clean_caption(original_caption: str) -> str:
    """
    Clean and extract the core description from the caption
    """
    # Remove common reasoning prefixes and introductory phrases
    prefixes_to_remove = [
        "i need to create a concise technical description",
        "i can't mention reasoning steps",
        "i should stick to describing",
        "the user wants",
        "i will focus on",
        "describing surgical scene"
    ]

    # Convert to lowercase for matching
    lower_caption = original_caption.lower()

    # Find the actual descriptive content
    for prefix in prefixes_to_remove:
        if prefix in lower_caption:
            # Split the caption and take the most substantive part
            parts = original_caption.split(prefix)
            if len(parts) > 1:
                caption = parts[-1].strip()
                break
    else:
        caption = original_caption

    # Remove any remaining reasoning language
    caption = re.sub(r'^(i|the model|it seems|from the image).+?:\s*', '', caption, flags=re.IGNORECASE)

    # Remove quotation marks and excessive whitespace
    caption = caption.strip('"').strip()

    return caption


def process_caption(frame_base64: str) -> str:
    """Generate clean, reasoning-free caption for a frame using OpenAI Vision API."""
    global client
    if client is None:
        raise HTTPException(status_code=500, detail="OpenAI client not initialized")

    try:
        # --- Step 1: Validate base64 image ---
        try:
            image_data = base64.b64decode(frame_base64)
            logging.info(f"Image decode successful. Length: {len(image_data)} bytes")
        except Exception as decode_error:
            logging.error(f"Base64 decoding error: {decode_error}")
            return "Invalid image data"

        # --- Step 2: Call OpenAI Vision API ---
        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # Use gpt-4o for vision capabilities
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": (
                                    # could work on this probably
                                    "Provide ONLY a concise, factual description of the contents of this surgical scene. "
                                    "Focus strictly on visible instruments, drapes, tubing, and equipment layout. "
                                    "DO NOT include any reasoning, self-comments, thoughts, or analysis steps. "
                                    "Output should be a final descriptive sentence or paragraph suitable for a caption, "
                                    "maximum 500 words. "
                                    "Avoid any markdown, headings, or formatting. Start directly with the description."
                                ),
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{frame_base64}"
                                },
                            },
                        ],
                    }
                ],
                max_tokens=1000,
            )

            logging.info(f"OpenAI Response received successfully")

        except Exception as api_error:
            logging.error(f"OpenAI API Error: {api_error}")
            return f"API error: {str(api_error)}"


        caption = ""
        try:
            if response.choices and len(response.choices) > 0:
                caption = response.choices[0].message.content.strip()
            else:
                logging.warning("No choices returned from OpenAI")
                return "No response generated"

        except Exception as extract_error:
            logging.error(f"Caption extraction error: {extract_error}")
            return "Failed to extract caption"


        caption = clean_caption(caption)

        logging.info(f"Final Clean Caption: {caption}")

        return caption if caption else "No meaningful caption generated"

    except Exception as e:
        logging.error(f"Unexpected error in caption generation: {e}")
        return f"Unexpected error: {e}"



@app.post("/process_frames")
async def process_frame(req: FrameRequest):
    """
    Unified endpoint for parallel processing of frame segmentation and captioning
    Stores everything in AWS RDS (no Pinecone)
    """
    try:
        # Generate session_id if not provided
        if not req.session_id:
            req.session_id = f"session-{req.patient_id}-{int(time.time())}"

        # Run segmentation and captioning in parallel
        segment_future = executor.submit(process_segment, req.frame_base64)
        caption_future = executor.submit(process_caption, req.frame_base64)

        # Wait for both to complete
        segment_result = segment_future.result()
        caption_result = caption_future.result()

        # Store in RDS only
        storage_result = store_frame_data(
            patient_id=req.patient_id,
            session_id=req.session_id,
            frame_base64=req.frame_base64,
            segmented_frame=segment_result['segmented_frame'],
            caption=caption_result
        )

        # Return results
        return {
            "analysis_id": storage_result["analysis_id"],
            "patient_id": req.patient_id,
            "session_id": req.session_id,
            "segmented_frame": segment_result['segmented_frame'],
            "caption": caption_result,
            "original_image_url": storage_result["original_url"],
            "segmented_image_url": storage_result["segmented_url"]
        }

    except Exception as e:
        print(f"Frame processing error: {e}")
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
# Add endpoint to retrieve analysis from RDS by analysis_id
@app.get("/get_analysis/{analysis_id}")
async def get_analysis(analysis_id: int):
    """
    Retrieve full analysis including caption from RDS
    """
    try:
        response = requests.get(
            f"{IMAGE_ANALYSIS_API}/{analysis_id}",
            timeout=10
        )

        if response.status_code == 200:
            return response.json()
        else:
            raise HTTPException(
                status_code=response.status_code,
                detail=f"Failed to retrieve analysis: {response.text}"
            )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


In [ ]:
# Query endpoint to search by patient or session
@app.get("/query_analyses")
async def query_analyses(
    patient_id: str = None,
    session_id: str = None,
    limit: int = 10
):
    """
    Query analyses from RDS by patient_id or session_id
    """
    try:
        params = {}
        if patient_id:
            params['patient_id'] = patient_id
        if session_id:
            params['session_id'] = session_id
        params['limit'] = limit

        response = requests.get(
            IMAGE_ANALYSIS_API,
            params=params,
            timeout=10
        )

        if response.status_code == 200:
            return response.json()
        else:
            raise HTTPException(
                status_code=response.status_code,
                detail=f"Failed to query analyses: {response.text}"
            )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))



In [ ]:


if __name__ == "__main__":
    import os

    # Initialize S3 Manager
    # add ur bucket name and AWS access creds
    s3_manager = S3ImageStorageManager(
        bucket_name='',
        aws_access_key_id="",
        aws_secret_access_key=""
    )

    # Initialize OpenAI client
    # add your openai KEY!
    initialize_openai(
        api_key="sk-proj"
    )

    # Ngrok setup
    # add your ngrok auth token!
    NGROK_AUTH_TOKEN = ""
    server_thread = setup_and_run_server(app, NGROK_AUTH_TOKEN, port=6111)

    # Optional: Keep the main thread alive
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nServer stopped.")

🌐 Public URL: NgrokTunnel: "https://3764327af39b.ngrok-free.app" -> "http://localhost:6111"

✅ Server is ready!

Use this URL in Unity:
   NgrokTunnel: "https://3764327af39b.ngrok-free.app" -> "http://localhost:6111"/segment

Test health check:
   NgrokTunnel: "https://3764327af39b.ngrok-free.app" -> "http://localhost:6111"/health


INFO:     Started server process [2713]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:6111 (Press CTRL+C to quit)


✅ Uploaded to S3: original/1763926741_78abe2de-43ce-4c00-875e-00eb2db71ad5.png
✅ Uploaded to S3: segmented/1763926742_7b402641-1bfd-4af8-9069-276756663043.png
✅ Stored in DB with analysis_id: 12
✅ Stored in RDS with analysis_id: 12
INFO:     174.114.222.197:0 - "POST /process_frames HTTP/1.1" 200 OK
✅ Uploaded to S3: original/1763926747_3bcc8988-129c-4550-8d4d-9405f24350b0.png
✅ Uploaded to S3: segmented/1763926747_9ce9ba9a-abe9-4cca-af8b-25b87a1c27e7.png
✅ Stored in DB with analysis_id: 13
✅ Stored in RDS with analysis_id: 13
INFO:     174.114.222.197:0 - "POST /process_frames HTTP/1.1" 200 OK
✅ Uploaded to S3: original/1763926752_5134d6fc-9ac1-403f-8c7e-8b88160fa48e.png
✅ Uploaded to S3: segmented/1763926752_cbac2b1e-9dfd-498a-b813-908d31a0b731.png
✅ Stored in DB with analysis_id: 14
✅ Stored in RDS with analysis_id: 14
INFO:     174.114.222.197:0 - "POST /process_frames HTTP/1.1" 200 OK
✅ Uploaded to S3: original/1763926757_074312c1-2364-4dcf-a664-53748c429e30.png
✅ Uploaded to S3: 